In [160]:
import aoc.common.process_input
import aoc.year2022.day19

In [161]:
fileContent = aoc.common.process_input.read_file("input\\year2022\\day19\\example.txt")        
input_list = aoc.common.process_input.to_function_list(fileContent, str)

In [162]:
blueprint1 = 'Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.'

def parse_blueprint(blueprint):
    # resources = (ore, clay, obsidian, geode)
    parts = blueprint.split(':')
    b_id = int(parts[0][10:])
    cost_parts = parts[1].strip().split('.')
    ore_robot_cost_parts = cost_parts[0].strip().split(' ')
    ore_robot_cost = int(ore_robot_cost_parts[4])
    clay_robot_cost_parts = cost_parts[1].strip().split(' ')
    clay_robot_cost = int(clay_robot_cost_parts[4])
    obsidian_robot_cost_parts = cost_parts[2].strip().split(' ')
    obsidian_robot_cost_ore = int(obsidian_robot_cost_parts[4])
    obsidian_robot_cost_clay = int(obsidian_robot_cost_parts[7])
    geode_robot_cost_parts = cost_parts[3].strip().split(' ')
    geode_robot_cost_ore = int(geode_robot_cost_parts[4])
    geode_robot_cost_obsidian = int(geode_robot_cost_parts[7])
    b = {}
    b['id'] = b_id
    b['ore-robot'] = (ore_robot_cost, 0, 0, 0)
    b['clay-robot'] = (clay_robot_cost, 0, 0, 0)
    b['obsidian-robot'] = (obsidian_robot_cost_ore, obsidian_robot_cost_clay, 0, 0)
    b['geode-robot'] = (geode_robot_cost_ore, 0, geode_robot_cost_obsidian, 0)

    return b_id, b

blueprint_id, blueprint = parse_blueprint(blueprint1)
print(blueprint_id, blueprint)

1 {'id': 1, 'ore-robot': (4, 0, 0, 0), 'clay-robot': (2, 0, 0, 0), 'obsidian-robot': (3, 14, 0, 0), 'geode-robot': (2, 0, 7, 0)}


In [163]:
# Each robot can collect 1 of its resource type per minute.
# It also takes one minute for the robot factory to construct any type of robot
# the factory consumes the necessary resources when construction begins

ore_robot = lambda n: (n, 0, 0, 0)
clay_robot = lambda n: (0, n, 0, 0)
obsidian_robot = lambda n: (0, 0, n, 0)
geode_robot = lambda n: (0, 0, 0, n)

def resource_add(r1, r2):
    ore = r1[0] + r2[0]
    clay = r1[1] + r2[1]
    obsidian = r1[2] + r2[2]
    geode = r1[3] + r2[3]
    r = (ore, clay, obsidian, geode)
    return r

def resource_can_sub(r1, r2):
    return r1[0] >= r2[0] and r1[1] >= r2[1] and r1[2] >= r2[2] and r1[3] >= r2[3]

def resource_sub(r1, r2):
    ore = r1[0] - r2[0]
    clay = r1[1] - r2[1]
    obsidian = r1[2] - r2[2]
    geode = r1[3] - r2[3]
    r = (ore, clay, obsidian, geode)
    return r    

class Factory:

    robot_template = {
        'ore-robot': ore_robot,
        'clay-robot': clay_robot,
        'obsidian-robot': obsidian_robot,
        'geode-robot': geode_robot
    }

    def __init__(self, bb):
        self.blueprint = bb

    def make_robot(self, robot_type, total_resources):
        # print(self.blueprint)
        resource_needed = self.blueprint[robot_type]
        if resource_can_sub(total_resources, resource_needed):
            remaining_resources = resource_sub(total_resources, resource_needed)
            return True, remaining_resources
        else:
            return False, total_resources

def get_next_state(state, try_create_robot, robot_factory):
    new_robot_list = []
    new_state = state_copy(state)
    resources = new_state['resources']
    
    if try_create_robot:
        can_make_robot, resources = robot_factory.make_robot('geode-robot', resources)
        if can_make_robot:
            new_state['geode'] += 1
        else:
            can_make_robot, resources = robot_factory.make_robot('obsidian-robot', resources)
            if can_make_robot:
                new_state['obsidian'] += 1
            else:
                if resources[3] == 0: # if I have geodes, don't make more clay robots
                    can_make_robot, resources = robot_factory.make_robot('clay-robot', resources)
                    if can_make_robot:
                        new_state['clay'] += 1
                    else:
                        can_make_robot, resources = robot_factory.make_robot('ore-robot', resources)
                        if can_make_robot:
                            new_state['ore'] += 1   

    resources = resource_add(resources, ore_robot(state['ore']))
    resources = resource_add(resources, clay_robot(state['clay']))
    resources = resource_add(resources, obsidian_robot(state['obsidian']))
    resources = resource_add(resources, geode_robot(state['geode']))
    
    new_state['resources'] = resources
    new_state['minute_counter'] += 1

    return new_state

def print_state(state):
    print('resources', state['resources'])
    print('ore', state['ore'], 'clay', state['clay'], 'obsidian', state['obsidian'], 'geode', state['geode'])

def are_equal_states(s1, s2):
    result = False
    if s1['resources'] == s2['resources']:
        if s1['ore'] == s2['ore'] and s1['clay'] == s2['clay']:
            if s1['obsidian'] == s2['obsidian'] and s1['geode'] == s2['geode']:
                result = True
    return result

def state_copy(s):
    copy_state = {}
    copy_state['minute_counter'] = s['minute_counter']
    copy_state['resources'] = s['resources']
    copy_state['ore'] = s['ore']
    copy_state['clay'] = s['clay']
    copy_state['obsidian'] = s['obsidian']
    copy_state['geode'] = s['geode']
    return copy_state

def get_max_needs(blueprint):
    needs = []
    needs.append(blueprint['ore-robot'])
    needs.append(blueprint['clay-robot'])
    needs.append(blueprint['obsidian-robot'])
    needs.append(blueprint['geode-robot'])
    needs_ore = [x[0] for x in needs]
    needs_clay = [x[1] for x in needs]
    needs_obsidian = [x[2] for x in needs]
    needs_geode = [x[3] for x in needs]
    max_ore = max(needs_ore)
    max_clay = max(needs_clay)
    max_obsidian = max(needs_obsidian)
    max_geode = max(needs_geode)
    return (max_ore, max_clay, max_obsidian, max_geode)


def prune_states(states, minute_counter, max_all_ores):
    state_list = states[minute_counter]
    new_state_list = []
    for state in state_list:
        # print_state(state)
        # keep only states with specific resource below max + 1 (except geode)
        # ore
        if state['resources'][0]  <= max_all_ores[0] + state['ore'] +1:
            # clay
            if state['resources'][1] <= max_all_ores[1] + state['clay'] +1:
                # obsidian        
                if state['resources'][2] <= max_all_ores[2] + state['obsidian'] +1:
                    new_state_list.append(state)
                    continue
            elif state['resources'][3] > 0:
                new_state_list.append(state)
                continue
    states[minute_counter] = new_state_list
    print('prune', len(state_list), len(new_state_list))

def is_state_in_state_list(state_list, state):
    for s in state_list:
        if are_equal_states(s, state):
            return True
    return False

In [164]:
blueprint_index, blueprint = parse_blueprint(blueprint1)
blueprint2 = 'Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.'
blueprint_index, blueprint = parse_blueprint(blueprint2)
robot_factory = Factory(blueprint)
robot_list = [ore_robot]
resources = (0, 0, 0, 0)

minute_counter = 0
minute_limit = 24

ore_robot_count = 1
clay_robot_count = 0
obsidian_robot_count = 0
geode_robot_count = 0

state = {}
state['minute_counter'] = minute_counter
state['resources'] = resources
state['ore'] = ore_robot_count
state['clay'] = clay_robot_count
state['obsidian'] = obsidian_robot_count
state['geode'] = geode_robot_count

states = {}
states[minute_counter] = [state]

max_all_needs = get_max_needs(robot_factory.blueprint)
print(max_all_needs)

# options for new state
# 1: try to make a new robot
# 2: do not try to make a new robot

while minute_counter < minute_limit:
    new_states = []
    
    for state in states[minute_counter]:
        ns_with_robot = get_next_state(state, True, robot_factory)
        ns_no_robot = get_next_state(state, False, robot_factory)

        if not is_state_in_state_list(new_states, ns_with_robot):
            new_states.append(ns_with_robot)
        if not are_equal_states(ns_with_robot, ns_no_robot):
            if not is_state_in_state_list(new_states, ns_no_robot):
                new_states.append(ns_no_robot)

    minute_counter += 1
    states[minute_counter] = new_states

    prune_states(states, minute_counter, max_all_needs)

for key in states:
    print(key)    
    for state in states[key]:
        print_state(state)

(3, 8, 12, 0)
prune 1 1
prune 1 1
prune 2 2
prune 3 3
prune 5 5
prune 10 8
prune 14 14
prune 22 19
prune 34 24
prune 44 38
prune 68 53
prune 105 65
prune 125 82
prune 142 101
prune 197 119
prune 224 120
prune 213 146
prune 279 173
prune 324 145
prune 266 137
prune 257 134
prune 233 166
prune 310 205
prune 333 176
0
resources (0, 0, 0, 0)
ore 1 clay 0 obsidian 0 geode 0
1
resources (1, 0, 0, 0)
ore 1 clay 0 obsidian 0 geode 0
2
resources (2, 0, 0, 0)
ore 1 clay 0 obsidian 0 geode 0
3
resources (1, 0, 0, 0)
ore 2 clay 0 obsidian 0 geode 0
resources (3, 0, 0, 0)
ore 1 clay 0 obsidian 0 geode 0
4
resources (3, 0, 0, 0)
ore 2 clay 0 obsidian 0 geode 0
resources (1, 0, 0, 0)
ore 1 clay 1 obsidian 0 geode 0
resources (4, 0, 0, 0)
ore 1 clay 0 obsidian 0 geode 0
5
resources (2, 0, 0, 0)
ore 2 clay 1 obsidian 0 geode 0
resources (5, 0, 0, 0)
ore 2 clay 0 obsidian 0 geode 0
resources (2, 1, 0, 0)
ore 1 clay 1 obsidian 0 geode 0
resources (2, 0, 0, 0)
ore 1 clay 1 obsidian 0 geode 0
resources (5,